In [2]:
from mistralai import Mistral
import yaml
import json
import requests 
from tqdm.auto import notebook_tqdm
from elasticsearch import Elasticsearch,  NotFoundError
import time
# from sentence_transformers import SentenceTransformer #embeddings
import pandas as pd

we are going to index the documents with the ids with elastic search. we are going to iterate over the question in our ground truth dataset (5 questions genrated for each question id)

for each q in question:
    execute q
    check if document is in the response

metrics: 
* Hit rate
* Mean reciprocal rank: the higher the position the relevant documents the better

In [3]:
with open('/workspaces/llm_search_engine/03_vector_search/docs_with_ids.json', 'rt') as f_in:
    documents = json.load(f_in)

In [18]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': 'c02e79ef'}

***PROBLEMS WITH ES***

- Tried prunnning evrything from my docker installation using: docker system prune --all --force --volumes
- Next, allocating cpu ram using this command: -e "ES_JAVA_OPTS=-Xms512m -Xmx512m", also memory lock to false: e "bootstrap.memory_lock=false"

```
docker run -it \
--rm \
--name elasticsearch \
-m 4GB \
-p 9200:9200 \
-p 9300:9300 \
-e "discovery.type=single-node" \
-e "xpack.security.enabled=false" \
docker.elastic.co/elasticsearch/elasticsearch:8.17.2
```

or 

```
docker run -it \
--rm \
--name elasticsearch \
-p 9200:9200 \  
-p 9300:9300 \
-e "discovery.type=single-node" \
-e "xpack.security.enabled=false" \
-e "ES_JAVA_OPTS=-Xms1g -Xmx1g" \
-e "bootstrap.memory_lock=false" \
-v /workspaces/llm_search_engine/elastic_data:/usr/share/elasticsearch/data \
docker.elastic.co/elasticsearch/elasticsearch:8.17.2
```

03/03/2025 --> last option worked as a charm but always prunning everything from my docker

In [5]:
es_client = Elasticsearch(
    hosts=[{'host': 'localhost', 'port': 9200, 'scheme': 'http'}],
    request_timeout=30,
    max_retries=10,
    retry_on_timeout=True
)

# Prueba la conexión
if es_client.ping():
    print("Conectado a Elasticsearch!")
else:
    print("No se pudo conectar a Elasticsearch.")

Conectado a Elasticsearch!


In [5]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
        }
    }
}

In [ ]:
# alias_name = "course-questions-metrics"

# new_index = f"{alias_name}_{int(time.time()*1000)}"

In [ ]:
# try:
#     es_client.indices.create(index=new_index, body=index_settings)
#     print(f"Índice '{new_index}' creado exitosamente.")
# except Exception as e:
#     print(f"Error creando el índice '{new_index}': {e}")

# actions = []
# old_indices = {}
# try:
#     old_indices = es_client.indices.get_alias(name=alias_name)
#     for idx in old_indices.keys():
#         actions.append({"remove": {"index": idx, "alias": alias_name}})
# except NotFoundError:
#     print(f"No existía el alias '{alias_name}', se creará de nuevo.")

# # Agregar la acción para añadir el alias al nuevo índice
# actions.append({"add": {"index": new_index, "alias": alias_name}})

# # Ejecutar la actualización de alias
# try:
#     es_client.indices.update_aliases(body={"actions": actions})
#     print(f"Alias '{alias_name}' ahora apunta al índice '{new_index}'.")
# except Exception as e:
#     print(f"Error actualizando el alias: {e}")

Error creando el índice 'course-questions-metrics_1740737517601': BadRequestError(400, 'resource_already_exists_exception', 'index [course-questions-metrics_1740737517601/P2vRCVe5Rsqu-YUNbxn58w] already exists')
Alias 'course-questions-metrics' ahora apunta al índice 'course-questions-metrics_1740737517601'.


In [6]:
index_name = "course-questions-metrics_index3"

if es_client.indices.exists(index=index_name):
    try:
        es_client.indices.delete(index=index_name, ignore_unavailable=True)
        print(f"Index {index_name} deleted successfully.")
    except Exception as e:
        print(f"Error deleting index {index_name}: {e}")

# Wait a short time before attempting to create the index again 
while es_client.indices.exists(index=index_name):
    print("Waiting for index to be fully deleted...")
    time.sleep(30)

try:
    es_client.indices.create(index=index_name, body=index_settings)
    print(f"Index {index_name} created successfully.")
except Exception as e:
    print(f"Error creating index {index_name}: {e}")

Index course-questions-metrics_index3 created successfully.


In [7]:
cluster_health = es_client.cluster.health()
print(cluster_health)

{'cluster_name': 'docker-cluster', 'status': 'green', 'timed_out': False, 'number_of_nodes': 1, 'number_of_data_nodes': 1, 'active_primary_shards': 1, 'active_shards': 1, 'relocating_shards': 0, 'initializing_shards': 0, 'unassigned_shards': 0, 'unassigned_primary_shards': 0, 'delayed_unassigned_shards': 0, 'number_of_pending_tasks': 0, 'number_of_in_flight_fetch': 0, 'task_max_waiting_in_queue_millis': 0, 'active_shards_percent_as_number': 100.0}


In [8]:
print(es_client.cat.indices(format="json"))

[{'health': 'green', 'status': 'open', 'index': 'course-questions-metrics_index3', 'uuid': 'gVNksAYkRZij-tLpxEo3Tw', 'pri': '1', 'rep': '0', 'docs.count': '0', 'docs.deleted': '0', 'store.size': '227b', 'pri.store.size': '227b', 'dataset.size': '227b'}]


In [ ]:
for doc in notebook_tqdm(documents):
    es_client.index(index=index_name, document=doc)
    # break

In [10]:
def elastic_search(query, course):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": course
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [11]:
elastic_search(
    query="I just discovered the course. Can I still join?",
    course="data-engineering-zoomcamp"
)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp',
  'id': '7842b56a'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp',
  'id': '63394d91'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it fin

In [13]:
df_ground_truth = pd.read_csv(r'/workspaces/llm_search_engine/03_vector_search/ground_truth_data.csv')

In [16]:
df_ground_truth.head()

,question,course,document
0,When does the course begin?,data-engineering-zoomcamp,c02e79ef
1,How can I get the course schedule?,data-engineering-zoomcamp,c02e79ef
2,What is the link for course registration?,data-engineering-zoomcamp,c02e79ef
3,How can I receive course announcements?,data-engineering-zoomcamp,c02e79ef
4,Where do I join the Slack channel?,data-engineering-zoomcamp,c02e79ef


In [14]:
ground_truth = df_ground_truth.to_dict(orient='records')

what im checking here is checking if the doc id is in the search results from elastic search. this means that the question id from the ground truth matches with the answer id in the documents. As a reminder, ES retrieves the questions - answers dic from the documents list and then we will match it with the ground truth dataset that has the id of the question with the answer

In [19]:
relevance_total = []

for q in notebook_tqdm(ground_truth):
    doc_id = q['document']
    results = elastic_search(query=q['question'], course=q['course'])
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

  0%|          | 0/4627 [00:00<?, ?it/s]

In [ ]:
relevance_total[0] # Here we see if the actual document has been retrieved and it which position

[True, False, False, False, False]

* hit-rate (recall): there is only 1 relevant docuemnt for each docuemnt. so we are going to look for at least one true in the relevance list
* MRR: looks at the true plus to the position. If true is in the first positoin we add 1, if it is second we add 1/2 ... we add 1/ rank, if none then it is 0

In [22]:
def hit_rate(relevance_total):
    cnt = 0
    for l in relevance_total:
        if True in l:
            cnt +=1

    return cnt / len(relevance_total)

In [24]:
def MRR(relevance_total):
    total_score = 0
    for l in relevance_total:
        for rank in range(len(l)):
            if l[rank]==True:
                total_score += 1/(rank+1)
    
    return total_score/len(relevance_total)

In [23]:
hit_rate(relevance_total)

0.7389237086665226

In [25]:
MRR(relevance_total)

0.6014624306606157

Next steps for this is to tweak the the parameters for ES, we could set a tracking environment so we could easily track what we change and what are the effects. Also the ground truth dataset was generated by a llm so probably it has mistakes. We could also use minsearch to evaluate or do the tracking so we have an alternative. 

## Vector Search

In [1]:
from sentence_transformers import SentenceTransformer #embeddings


In [ ]:
model = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/11.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/383 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

**We are going to embed question, answers and questions and answers**

In [8]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_vector":{"type":"dense_vector", "dims":384, "index":True, "similarity":"cosine"},
            "text_vector":{"type":"dense_vector", "dims":384, "index":True, "similarity":"cosine"},
            "text_question_vector":{"type":"dense_vector", "dims":384, "index":True, "similarity":"cosine"},

        }
    }
}

In [9]:
index_name = "vector_indices_complete"

if es_client.indices.exists(index=index_name):
    try:
        es_client.indices.delete(index=index_name, ignore_unavailable=True)
        print(f"Index {index_name} deleted successfully.")
    except Exception as e:
        print(f"Error deleting index {index_name}: {e}")

# Wait a short time before attempting to create the index again 
while es_client.indices.exists(index=index_name):
    print("Waiting for index to be fully deleted...")
    time.sleep(30)

try:
    es_client.indices.create(index=index_name, body=index_settings)
    print(f"Index {index_name} created successfully.")
except Exception as e:
    print(f"Error creating index {index_name}: {e}")

Index vector_indices_complete created successfully.


In [10]:
for doc in notebook_tqdm(documents):
    question = doc['question']
    text = doc['text']
    qt = question + ' ' + text

    doc['question_vector'] = model.encode(question)
    doc['question_text'] = model.encode(text)
    doc['question_qt'] = model.encode(qt)

    es_client.index(index=index_name, document=doc)

  0%|          | 0/946 [00:00<?, ?it/s]

In [15]:
search_term = "can i still join the course?"
vec_search = model.encode(search_term)

In [19]:
query = {
    "field" : "question_vector", #where to look
    "query_vector": vec_search,
    "k": 5,#nb of responess
    "num_candidates":10000
}

In [20]:
es_res = es_client.search(index=index_name,
                            knn=query,
                            source=["text", "section", "question", "course", "id"], explain=True) #scoring functions for scoring can be custom made


In [21]:
es_res

ObjectApiResponse({'took': 100, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 5, 'relation': 'eq'}, 'max_score': 0.95504403, 'hits': [{'_shard': '[vector_indices_complete][0]', '_node': '25mhkuJzSHKluCmZkuL3UA', '_index': 'vector_indices_complete', '_id': '0Z0DXZUBY6gVS3bpat7p', '_score': 0.95504403, '_source': {'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'section': 'General course-related questions', 'question': 'The course has already started. Can I still join it?', 'course': 'machine-learning-zoomcamp', 'id': 'ee58a693'}, '_explanation': {'value': 0.95504403, 'description': 'wit

we are going to filter out the results and improve the es query plus we are going to define a function so we use the complex query plus a selection of the filters that we are going to use.

The thing that we have to take into account here is that vectorizing parts of the query or all of it + filtering or combining everything together.

Checking and comparing several results tracking parameters that have changed and all that is the way to evaluate the performance on the search



In [22]:
def elastic_search_knn(field, vector, course):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [ ]:
def text_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn('text_vector', v_q, course)

In [ ]:
def elastic_search_knn_combined(vector, course):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": [
                    {
                        "script_score": {
                            "query": {
                                "term": {
                                    "course": course
                                }
                            },
                            "script": {
                                "source": """
                                    cosineSimilarity(params.query_vector, 'question_vector') + 
                                    cosineSimilarity(params.query_vector, 'text_vector') + 
                                    cosineSimilarity(params.query_vector, 'question_text_vector') + 
                                    1
                                """,
                                "params": {
                                    "query_vector": vector
                                }
                            }
                        }
                    }
                ],
                "filter": {
                    "term": {
                        "course": course
                    }
                }
            }
        },
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [ ]:
def vector_combined_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn_combined(v_q, course)

# evaluate(ground_truth, vector_combined_knn)